In [28]:
from clickhouse_driver import Client
from datetime import date, datetime, timedelta
import pandas
import geopandas
import os

In [29]:
CLICKHOUSE_HOST=os.getenv("CLICKHOUSE_HOST")
CLICKHOUSE_USERNAME=os.getenv("CLICKHOUSE_USERNAME")
CLICKHOUSE_PASSWORD=os.getenv("CLICKHOUSE_PASSWORD")
client = Client(CLICKHOUSE_HOST, user=CLICKHOUSE_USERNAME, password=CLICKHOUSE_PASSWORD)

In [30]:
%%time

data, cols = client.execute("""
select * from weatherStationObservation
 JOIN WeatherStation_postgres
 ON WeatherStation_postgres.id = weatherStationObservation.station_id
""", with_column_types=True)

df = pandas.DataFrame(data, columns=[col_name for col_name, _ in cols])

CPU times: user 16.2 ms, sys: 3.55 ms, total: 19.8 ms
Wall time: 86.5 ms


In [33]:
gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.lon, df.lat), crs="EPSG:4326")
gdf['measurement_moment'] = df['measurement_moment'].dt.strftime('%Y-%m-%d %H:%M:%S')
gdf.explore("station_name", legend=True)